In [299]:
# Necessary imports
import numpy as np
import pandas as pd
import pickle
import random
pd.set_option('display.max_colwidth', -1)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

## Firstly, read in all the things necessary to run the model here

In [3]:
# Read in dataframe of clean dishes

cleaned_data = pd.read_pickle('data/kaggle_and_reddit_dishes_no_spaces.pkl')

In [7]:
# Read in seeds

with open('flask/flask_app/pickles/patterns.pkl', 'rb') as f:
    seeds = pickle.load(f)

In [9]:
# Read in unique characters

with open('flask/flask_app/pickles/unique_chars.pkl', 'rb') as f:
    chars = pickle.load(f)

In [10]:
# Read in character to integer dictionary

with open('flask/flask_app/pickles/char_to_int.pkl', 'rb') as f:
    char_to_int = pickle.load(f)

In [11]:
# Define and load model

seq_len = 35
n_vocab = 51

model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, n_vocab)))
model.add(Dropout(0.2))
model.add(Dense(n_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

filepath_current = "flask/flask_app/pickles/weights-improvement-150-0.5860.hdf5"
model.load_weights(filepath_current)
model.compile(loss='categorical_crossentropy', optimizer='adam')


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Define functions to run model

In [12]:
# define helper function to adjust by temperature:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    np.seterr(divide = 'ignore') 
    return np.argmax(probas)

In [186]:
# define function to generate text
def generate_dish(seeds, lstm_model, temp, seq_len, n_vocab, output_len,
                 char_to_int_dict, chars):
    
    """
    Generate a dish based on a series of seeds and an LSTM model

    seeds: list of strings (each being seq_length long)
    lstm_model: model, the LSTM model to use
    temp: int, the temperature hyperparameter for LSTM
    seq_len: int, the length of the seed string
    n_vocab: int, number of unique characters
    output_len: int, length of string to output (number of characters to generate)
    char_to_int_dict: dict to convert characters to integers
    chars: list, sorted set of all possible characters
    """

    # Select a random pattern from seeds (the list of all patterns) 
    # as a random seed
    start_index = np.random.randint(0, len(seeds))
    orig_seed = seeds[start_index]

    # initialize dish as empty list
    dish = []
    seed = orig_seed

    # generate characters one by one
    for w in range(output_len):
        sampled = np.zeros((1, seq_len, n_vocab))
        for t, char in enumerate(seed):
            sampled[0, t, char_to_int_dict[char]] = 1.


        preds = lstm_model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temp)
        next_char = chars[next_index]

        seed += next_char
        seed = seed[1:]

        dish.append(next_char)

    dish_gen = "".join(dish)
    return dish_gen

In [384]:
output = generate_dish(seeds=seeds, lstm_model=model, temp=0.7, seq_len=seq_len, 
                       n_vocab=n_vocab, output_len=100, char_to_int_dict=char_to_int, chars=chars)
print(output)

erved sweet potato chopsed in the cover and with meat or fish orders wide dotuto sauce steamed rice 


Ok how the hell are we gonna filter this thing

In [241]:
output

'uce toasted baked potato peach chilli hot sour saute fill things mashed potatoes garlic mayonnaise d'

In [37]:
# get rid of half-words on the ends
' '.join(output.split(' ')[1:-1])

'biscuit compote grapefruit old sauce french fried potatoes or sauce with cheese served on a'

In [47]:
# Get a list of all words in my corpus

all_text = ''

for index, row in cleaned_data.iterrows():
    all_text += row['cleaned_title'] + ' '

In [48]:
all_words = all_text.split(' ')

In [53]:
unique_words = list(set(all_words))

In [354]:
# pickle unique words:
with open('flask/flask_app/pickles/unique_words.pkl', 'wb') as f:
    pickle.dump(unique_words, f)

In [55]:
len(all_words)

283711

In [56]:
len(unique_words)

21766

### Cleaning function

In [344]:
def clean_output(raw_dish, avail_words):
    """
    function to clean the output of my LSTM
    """
    print(raw_dish)   
    
    bad_words = ['the', 'potat', 'nd', 'ad', 'comments', 'look', 'potatoe', 'read', 'er', 'us', 'review']
    filler_words = ['and', 'of', 'with', 'or', 'a', 'choice', 'of', 'served', 'on', 'cooked', 'in', 'from']
    
    word_list = raw_dish.strip().split(' ')[1:]
    cleaned_dish_list = []
    
    # Exclude any words not in the original corpus and in the "bad words list"
    # EXCEPT, I allow one typo to make things more interesting
    typos = 0
    for word in word_list:
        if word in avail_words and word not in bad_words:
            cleaned_dish_list.append(word)
        elif word not in avail_words:
            if typos < 1:
                cleaned_dish_list.append(word)
            typos += 1
    
    # Limit to only one and/one or/one sauce:
    num_and, num_or, num_sauce, num_potato, num_cheese = 0, 0, 0, 0, 0
    sauce_lim = random.randint(1,2)
    potato_lim = random.randint(1,2)
    cheese_lim = random.randint(1,2)
    
    for i, word in enumerate(cleaned_dish_list):
        if word == 'and':
            num_and += 1
        elif word == 'or':
            num_or += 1
        elif word == 'sauce':
            num_sauce += 1
        elif word == 'potato' or word == 'potatoes':
            num_potato += 1
        elif word == 'cheese':
            num_cheese += 1
        if num_and > 1 or num_or > 1 or num_sauce > sauce_lim or \
            num_potato > potato_lim or num_cheese > cheese_lim:
            cleaned_dish_list = cleaned_dish_list[:i]
    
    # Make sure the dish doesn't end with filler words
    while cleaned_dish_list[-1] in filler_words or len(cleaned_dish_list[-1]) <= 2:
        cleaned_dish_list = cleaned_dish_list[:-1]
    
    # Make sure the dish doesn't start with filler words
    while cleaned_dish_list[0] in filler_words:
        cleaned_dish_list = cleaned_dish_list[1:]
        
    print(' '.join(cleaned_dish_list))

In [385]:
clean_output(output, unique_words)

erved sweet potato chopsed in the cover and with meat or fish orders wide dotuto sauce steamed rice 
sweet potato chopsed in cover and with meat or fish orders wide sauce steamed rice
